In [2]:
!pip install fastapi uvicorn pyngrok python-multipart pillow pytesseract
!apt-get install -y tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [74]:
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import JSONResponse
from PIL import Image, ImageEnhance
import pytesseract
import io
import re

app = FastAPI()

# Utility: Check if test_value is out of reference range
def is_out_of_range(value_str, ref_range_str):
    try:
        value = float(value_str)
        if "-" in ref_range_str:
            low, high = map(float, ref_range_str.split("-"))
            return value < low or value > high
        elif "<" in ref_range_str:
            limit = float(ref_range_str.replace("<", ""))
            return value > limit
        elif ">" in ref_range_str:
            limit = float(ref_range_str.replace(">", ""))
            return value < limit
    except ValueError:
        return None


@app.post("/get-lab-tests")
async def get_lab_tests(file: UploadFile = File(...)):
    # ... (image loading and preprocessing)

    # Extract text using OCR
    raw_text = pytesseract.image_to_string(image, config='--psm 6')

    # Print raw OCR output (NEW LINE)
    print("Raw OCR Output:", raw_text)

    # Try to extract test results: name, value, range
    # Updated regex pattern (NEW LINE)
    test_pattern = r"([A-Za-z\s\(\)]+)\s*([\d\.]+(?:\s*[a-zA-Z]+)?)\s*(?:([\d\.]+\s*-\s*[\d\.]+)|([\d\.]+\s*[<>]\s*[\d\.]+))?"

    results = []
    for match in re.findall(test_pattern, raw_text):
        name, value, ref_range1, ref_range2 = match
        ref_range = ref_range1 or ref_range2
        out_of_range = is_out_of_range(value, ref_range) if ref_range else None  # Handle missing ref_range
        unit = "g/dL" if "HB" in name else "%"  # crude heuristic
        results.append({
            "test_name": name.strip(),
            "test_value": value,
            "bio_reference_range": ref_range if ref_range else "N/A",  # Handle missing ref_range
            "test_unit": unit,
            "lab_test_out_of_range": out_of_range
        })

    return JSONResponse(content={
        "is_success": True,
        "data": results
    })

In [75]:
!ngrok config add-authtoken 2wOGSRH75GVXyuTzutsJVjEflEg_31g3DMcpTbkr6YHTKGnYj

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [83]:
# Kill all existing ngrok tunnels
for tunnel in ngrok.get_tunnels():
    ngrok.disconnect(tunnel.public_url)

In [84]:
from pyngrok import ngrok
import uvicorn
import threading

# Open a tunnel to the FastAPI app
public_url = ngrok.connect(8000)
print("FastAPI app is live at:", public_url)

def run():
    uvicorn.run(app, host="0.0.0.0", port=8000)

FastAPI app is live at: NgrokTunnel: "https://83c0-104-198-14-253.ngrok-free.app" -> "http://localhost:8000"


In [85]:
import requests

# Replace with your ngrok URL and correct endpoint
url = "https://83c0-104-198-14-253.ngrok-free.app/get-lab-tests"

# Open the image file to send
files = {'file': open("/content/AHD-0425-PA-0008061_E-mahendrasinghdischargecard_250427_1114@E.pdf_page_13.png", "rb")}

# Send the POST request
response = requests.post(url, files=files)

# Print the response from the FastAPI app
print(response.json())


INFO:     104.198.14.253:0 - "POST /get-lab-tests HTTP/1.1" 200 OK
{'is_success': True, 'data': []}
